<H1>Visualization of Demo Output</H1>
<H2>Preliminaries</H2>
This notebook visualizes data generated by successfully running the <a href="https://confluence.lsstcorp.org/display/LSWUG/Testing+the+Installation">LSST Demo</a>. It also assumes you have loaded the LSST environment and have defined an environment variable (prior to starting this notebook) for the demo `/output` directory. 

The demo will have generated a set of calibrated images (`calexp`'s), one for each filter, as well as catalogs of sources detected in each image. The results will be stored in the output repository (i.e., the `/output` directory and sub-directories). The processed images are from the following SDSS fields (filters in parentheses are not processed by `demo_small.sh`): 
Run  | Camcol | Field | Filters
---- |:------:| ----- | -------
4192 | 4      | 300   | (ur)giz
6377 | 4      | 399   | (gz)uri

This exercise illustrates the capabilities in the LSST **afw** package for displaying images in **ds9** and visualizing the sources that were detected by the demo processing script. 

<h2>Visualization of Detected Sources</h2>
The code segments below should be executed in order within this notebook (use the "play" button in the GUI to execute the code cells). Alternatively, if you plan to work in a live shell (rather than the iPython notebook), start a (i)python session and cut/paste to execute the code segments. 

Starb by importing some necessary packages:

In [1]:
import os
import lsst.daf.persistence  as dafPersist
import lsst.afw.image        as afwImage
import lsst.afw.display.ds9  as ds9

We will first display the output *r*-band image (**ds9** will be started for you if needed). 

In [2]:
# Create a butler instance to provide access to the data
DATA_DIR = os.environ.get('DATA_DIR')
butler = dafPersist.Butler(DATA_DIR)

# Fetch the r-band exposure for the specified run/field/camcol/filter
dataId = dict(run=6377, field=399, filter='r', camcol=4)
exposure = butler.get('calexp', dataId)

# display the exposure, with mask overlay
settings = {'scale':'linear', 'zscale':'', 'zoom': 1, 'mask' : 'transparency 40'}
title = "SDSS r-band; Run: 6377, Field: 399"
# Need to have ds9 running, otherwise the following fails.ds9.mtv(exposure, frame=1, title=title, settings=settings)

ds9 doesn't appear to be running (XPA: XPA returned NULL, (;iconify no; raise)), I'll exec it for you


RuntimeError: 
  File "python/lsst/afw/display/simpleFits.cc", line 466, in void lsst::afw::display::writeBasicFits(int, const ImageT &, const image::Wcs *, const char *) [ImageT = lsst::afw::image::Image<float>]
    Error writing data for row 22 {0}
lsst::pex::exceptions::RuntimeError: 'Error writing data for row 22'


Note that the mask image has been superimposed with a 40% transparancy: it shows the location of bad pixels (green denotes a bad column of saturated pixels) and detections (in blue). The color of masked pixels denotes their type:

Use the **butler** to fetch all detected sources from the output catalog, and overplot circles on the displayed image at the source locations. *Note that the sources have not been filtered using quality flags.*  

In [3]:
# Fetch the sources
sources = butler.get('src', dataId)

# Overplot circles at the locations of detected sources
symbol = "o"
with ds9.Buffering():
    for i,source in enumerate(sources):
        ds9.dot(symbol, source.getX(), source.getY(), ctype=ds9.WHITE, size=5, frame=1, silent=True)

The entire visualization above can be extended with a little more python to show all sources in all bands. The following script will display each specified band in its own frame, along with the detected sources on that image. 

In [4]:
FILTER_MAP = {'u': 0, 'g':1, 'r': 2, 'i': 3, 'z': 4, 'y': 5}
MARKER_COLORS = (ds9.BLUE, ds9.GREEN, ds9.RED, ds9.YELLOW, ds9.WHITE)
MARKER_SIZES  = (4, 5, 6, 7, 8)

def demoDisplay(rootDir, run, field, filters='ugriz', title="", 
                scale="linear", trans=40, useEllipse=False):

    # put the display settings in a dict object 
    settings = {'scale':scale, 'zscale':'', 'zoom': 1, 'mask' : 'transparency %d' %(trans)}

    # make a butler for retrieving exposures
    butler = dafPersist.Butler(rootDir)

    # Display the exposure in each filter, and plot the detected sources
    for j,filter in enumerate(filters):

        # Fetch the exposure from the run/field/filter/camcol:
        dataId = dict(run=run, field=field, filter=filter, camcol=4)
        exposure = butler.get('calexp', dataId)
        idx = FILTER_MAP[filter]

        # Display the exposure with ds9.mtv()
        if not title:
            frameTitle = "SDSS %s-band; Run: %d, Field: %d" % (filter, run, field)
        else:
            frameTitle = title
        ds9.mtv(exposure, frame=j+1, title=frameTitle, settings=settings)

        # Now get the catalog and over-plot the sources
        sources = butler.get('src', dataId)
        with ds9.Buffering():
            for i,source in enumerate(sources):
                if useEllipse:
                    # show an ellipse symbol
                    symbol = "@:{ixx},{ixy},{iyy}".format(ixx=source.getIxx(),
                                                      ixy=source.getIxy(),
                                                      iyy=source.getIyy())
                else:
                    # just a simple point (symbols +, x, *, o are all accepted)
                    symbol = "o"
                
                ds9.dot(symbol, source.getX(), source.getY(), ctype=MARKER_COLORS[idx],
                    size=MARKER_SIZES[idx], frame=j+1, silent=True)
        
demoDisplay(os.environ.get('DATA_DIR'), 6377, 399, filters='ugriz')

Try experimenting with the code, perhaps by plotting sources from all bands on a single image. See <a href="http://lsst-web.ncsa.illinois.edu/doxygen/x_masterDoxyDoc/afw_sec_display.html">Using DS9 in the LSST Framework</a> for additional details. 

<font size=-2>The above script was adapted from showInDs9.py, written by S. Bickerton of the HSC Software Group.</font>